In [1]:
import numpy as np
import os 
from pathlib import Path
import glob
import matplotlib.pyplot as plt
import sys
sys.path.append('../')
sys.path.append('../../')
from dataset import get_paths, PathLike, is_image
import pandas as pd
import shutil
from dataset.path_utils import get_paths_from_dirs

In [2]:
    # на тестовом датесете с разной разметкой выполнить предсказания и вывести метрики
os.listdir(dataset_root)

NameError: name 'dataset_root' is not defined

In [55]:
path_2_test = Path('../../../dataset/segmentation/test/')

dataset_root = "/home/kirilman/Project/dataset/segmentation/test.yaml"
path_2_nets  = {'small': Path('../../../asbestos/yolov5/runs/train-seg/small_l_b4_512/weights/best.pt'),
                'large': Path('../../../asbestos/yolov5/runs/train-seg/large_l_b4_512/weights/best.pt'),
                'gener': Path('../../../asbestos/yolov5/runs/train-seg/m_batch4_imgs512/weights/best.pt'),
                'generl':Path('../../../asbestos/yolov5/runs/train-seg/l_batch4_imgs512/weights/best.pt') }

path_2_dataset = {'small': Path('../../../dataset/segmentation/labels/small/test/'),
                  'large': Path('../../../dataset/segmentation/labels/large_segment/test/'),
                  'gener': Path('../../../dataset/segmentation/labels/original/test/'),}

def copy(_from, to):
    files = os.listdir(_from)
    files = list(filter( lambda x:False if is_image(x) else True, files))
    for file in files:
        shutil.copy(_from / file, to / file)
                    


In [56]:
for name, path_2_data in path_2_dataset.items():
    print(name)

small
large
gener


In [57]:
for n, path_2_data in path_2_dataset.items():
    copy(path_2_data, path_2_test)
    for name, path_2_net in path_2_nets.items():
        print('dataset:*',n,'*', path_2_net,'*')
        !python ../../yolov5/segment/val.py --data {dataset_root} --weight {path_2_net} --project ./result/ --imgsz 512 --batch-size 1 --name {name} > r.txt
        print('-------------------------------------------')
        

dataset:* small * ../../../asbestos/yolov5/runs/train-seg/small_l_b4_512/weights/best.pt *
segment/val: data=/home/kirilman/Project/dataset/segmentation/test.yaml, weights=['../../../asbestos/yolov5/runs/train-seg/small_l_b4_512/weights/best.pt'], batch_size=1, imgsz=512, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=./result/, name=small, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.2-216-g6e544d5f Python-3.9.12 torch-1.11.0+cu113 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 5947MiB)

Fusing layers... 
Model summary: 275 layers, 47474367 parameters, 0 gradients, 146.4 GFLOPs
val: Scanning '/home/kirilman/Project/dataset/segmentation/test' images and labe
val: New cache created: /home/kirilman/Project/dataset/segmentation/test.cache
                 Class     Images  Instances      Box(P          R      mAP50  m
                 

In [112]:
with open('r.txt','r') as f:
    data = f.read()

In [118]:
result = []
for l in data.split('\n'):
    if l[:3] == 'dat':
        dataset = l.split('*')[1]
        name_net = l.split('*')[2]
    if 'asbest' in l:
        line_res = l
    if l[:3] == '---':
        r = dataset+'|'+name_net+'|'+line_res
        t =list(filter(lambda x: True if len(x)>1 else False, r.split(' ')))
        r = {'dataset':t[0],'net':t[1],'mAP50':t[-2], 'mAP50-95':t[-1]}
        result.append(r)

In [125]:
for r in result:
    r['net'] = r['net'].split('/')[-3]

In [126]:
pd.DataFrame(result)

,dataset,net,mAP50,mAP50-95
0,small,small_l_b4_512,0.576,0.335
1,small,large_l_b4_512,0.191,0.115
2,small,m_batch4_imgs512,0.516,0.29
3,small,l_batch4_imgs512,0.511,0.294
4,large,small_l_b4_512,0.076,0.0504
5,large,large_l_b4_512,0.587,0.388
6,large,m_batch4_imgs512,0.357,0.233
7,large,l_batch4_imgs512,0.416,0.284
8,gener,small_l_b4_512,0.492,0.287
9,gener,large_l_b4_512,0.394,0.253


In [111]:
import subprocess
with open('t.txt','w') as f:
    p = subprocess.Popen(['python',"run.py"],stdout = f)
    p.wait()
    print(p.stdout)

None


In [146]:
!pwd

/home/kirilman/Project/asbestos/notebooks/visualize_result


In [ ]:
models = get_paths_from_dirs(['../../yolov5/runs/train-seg/'],['pt'])
models = list(filter(lambda x: x.name.split('/')[-1] == 'best.pt', models))
print(len(models), models[-1])
path_test_yml = Path('../../../dataset/segmentation/test.yaml')
copy(Path('../../../dataset/segmentation/labels/original/test/'), Path('../../../dataset/segmentation/test/'))
for path_2_model in models:
    name = str(models[0]).split('/')[-3]
    !python ../../yolov5/segment/val.py --data {path_test_yml} --weight {path_2_model} --imgsz 512 --batch-size 1 --name {name} > r.txt
    print('-----\n')

In [248]:
result = []
import re
with open('r.txt','r') as f:
    data = f.read()
for l in data.split('\n'):
    if l[:11] == 'segment/val':
        a = re.search("\[.",l)
        a = a.start() + 2
        b = re.search("\]",l)
        b = b.start() - 1
        model = l[a:b].split('/')[-3]
    if 'asbest' in l:
        t =list(filter(lambda x: True if len(x)>1 else False, l.split(' ')))
        r = { 'net':model,'P':t[-4], 'R':t[-3], 'mAP50':t[-2], 'mAP50-95':t[-1]}
        result.append(r)
pd.DataFrame(result)

,net,P,R,mAP50,mAP50-95
0,large_l_b4_512,0.762,0.271,0.36,0.232
1,x_512_batch_1@,0.654,0.616,0.628,0.383
2,exp2,0.566,0.631,0.598,0.281
3,l_batch4_imgs512,0.706,0.613,0.65,0.394
4,exp,0.442,0.438,0.403,0.158
5,merge_416_b2,0.389,0.388,0.251,0.0786
6,m_batch4_imgs512,0.66,0.586,0.634,0.373
7,l_512_batch_2,0.682,0.682,0.703,0.436
8,m_512_b4@,0.651,0.62,0.619,0.375
9,exp3,0.675,0.631,0.695,0.382


In [3]:
result = []
import re
with open('result_val.txt','r') as f:
    data = f.read()
for l in data.split('\n'):
    if l[:11] == 'segment/val':
        a = re.search("\[.",l)
        a = a.start() + 2
        b = re.search("\]",l)
        b = b.start() - 1
        model = l[a:b].split('/')[-3]
    if 'rocks' in l:
        t =list(filter(lambda x: True if len(x)>1 else False, l.split(' ')))
        r = { 'net':model,'P':t[-4], 'R':t[-3], 'mAP50':t[-2], 'mAP50-95':t[-1]}
        result.append(r)
pd.DataFrame(result)

,net,P,R,mAP50,mAP50-95
0,m_batch4_imgs512,0.704,0.0774,0.386,0.279
1,x_batch4_imgs512,0.601,0.158,0.395,0.279
2,small_l_b4_512,0.662,0.156,0.408,0.272
3,x_b8_img640,0.776,0.181,0.468,0.339
4,large_l_b4_512,0.797,0.26,0.515,0.376
5,all_l_b4_512,0.733,0.0876,0.407,0.299
6,merge_l_b4_512,0.672,0.639,0.652,0.38
7,merge_filter_small_l_b4_512,0.703,0.601,0.668,0.404
8,merge_filter_small_l_b4_512_flip,0.703,0.601,0.668,0.404
9,merge_filter_small_l_b4_512_flip2,0.704,0.629,0.708,0.424


In [7]:
result = []
import re
with open('./result/result_val_origin.txt','r') as f:
    data = f.read()
for l in data.split('\n'):
    if l[:11] == 'segment/val':
        a = re.search("\[.",l)
        a = a.start() + 2
        b = re.search("\]",l)
        b = b.start() - 1
        model = l[a:b].split('/')[-3]
    if ' all' in l:
        t =list(filter(lambda x: True if len(x)>1 else False, l.split(' ')))
        r = { 'net':model,'P':t[-4], 'R':t[-3], 'mAP50':t[-2], 'mAP50-95':t[-1]}
        result.append(r)
pd.DataFrame(result)

,net,P,R,mAP50,mAP50-95
0,m_batch4_imgs512,0.662,0.585,0.634,0.373
1,x_batch4_imgs512,0.687,0.628,0.684,0.412
2,small_l_b4_512,0.595,0.454,0.483,0.282
3,x_b8_img640,0.712,0.622,0.654,0.4
4,large_l_b4_512,0.764,0.271,0.36,0.232
5,all_l_b4_512,0.701,0.613,0.649,0.394
6,merge_l_b4_512,0.606,0.0981,0.346,0.225
7,merge_filter_small_l_b4_512,0.737,0.212,0.475,0.337
8,merge_filter_small_l_b4_512_flip,0.737,0.212,0.475,0.337
9,merge_filter_small_l_b4_512_flip2,0.563,0.111,0.341,0.244
